In [3]:
import re, time
import pandas as pd
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack import Pipeline, Document

import os
from getpass import getpass


/home/sanka/anaconda3/envs/tf-env-ultimate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-19 11:22:27.976886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 11:22:28.930789: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def process_log_file(file_path)-> pd:
    pattern = r'(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) \((?P<agent>[^)]+)\) (?P<debug>[A-Za-z_\.]+) (?P<level>[A-Z]+): (?P<message>.*)'

    data = {
        "timestamp": [],
        "agent": [],
        "debug": [],
        "level": [],
        "message": []
    }

    with open(file_path, 'r') as file:
        current_log = []
        line_count = 0

        for line in file:
            # Check if the line matches the start of a new log entry
            if re.match(pattern, line):
                if current_log:
                    full_log_entry = "\n".join(current_log)
                    match = re.match(pattern, full_log_entry, re.DOTALL)
                    if match:
                        data["timestamp"].append(match.group('timestamp'))
                        data["agent"].append(match.group('agent'))
                        data["debug"].append(match.group('debug'))
                        data["level"].append(match.group('level'))
                        data["message"].append(match.group('message').strip())

                current_log = [line.strip()]
                line_count = 1
            else:
                if line_count < 4:
                    current_log.append(line.strip())
                    line_count += 1

        # Append the last log entry
        if current_log:
            full_log_entry = "\n".join(current_log)
            match = re.match(pattern, full_log_entry, re.DOTALL)
            if match:
                data["timestamp"].append(match.group('timestamp'))
                data["agent"].append(match.group('agent'))
                data["debug"].append(match.group('debug'))
                data["level"].append(match.group('level'))
                data["message"].append(match.group('message').strip())

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    return df

In [5]:
log=process_log_file('volttron.log')
print(log.head)

<bound method NDFrame.head of                       timestamp                                  agent  \
0       2024-06-17 15:47:34,176  platform_driveragent-4.0 878936 [103]   
1       2024-06-17 15:47:34,180  platform_driveragent-4.0 878936 [702]   
2       2024-06-17 15:47:34,185         listeneragent-3.3 878937 [103]   
3       2024-06-17 15:47:34,188          listeneragent-3.3 878937 [63]   
4       2024-06-17 15:47:34,188         listeneragent-3.3 878937 [702]   
...                         ...                                    ...   
206630  2024-06-19 11:22:40,001  platform_driveragent-4.0 878936 [223]   
206631  2024-06-19 11:22:40,001  platform_driveragent-4.0 878936 [227]   
206632  2024-06-19 11:22:40,001  platform_driveragent-4.0 878936 [288]   
206633  2024-06-19 11:22:40,004  platform_driveragent-4.0 878936 [294]   
206634  2024-06-19 11:22:40,007          listeneragent-3.3 878937 [99]   

                                   debug  level  \
0                             

In [2]:
def split_sentence_with_overlap(sentence, n, m):
    # Split the sentence into words
    words = sentence.split()
    
    # Initialize the list to hold the chunks
    chunks = []
    
    # Use a sliding window approach to create chunks with overlap
    for i in range(0, len(words), n - m):
        chunk = words[i:i + n]
        if len(chunk) == n:
            chunks.append(' '.join(chunk))
        else:
            # Handle the case where the last chunk might be smaller than n words
            chunks.append(' '.join(chunk))
            break
    
    return chunks

In [18]:
from haystack.utils import Secret

no_of_rows=100
# Create a list to store the context and prompts
data = []
client = OpenAIGenerator(model="gpt-3.5-turbo",generation_kwargs={"max_tokens": 4000}, api_key=Secret.from_token(""))
# Convert each row to a JSON object suitable for ChatGPT prompts
current_context=f"\n"
prompt=''
row_count=0
for _, row in log.iterrows():
    context = f"Timestamp: {row['timestamp']}\nAgent: {row['agent']}\nDebug: {row['debug']}\nLevel: {row['level']}\nMessage: {row['message']}"
    if row_count<=no_of_rows:
        current_context=current_context+f"\n"+context
        
    else:
       # prompt = f"provide a detaled technical report of the system for the system log in this given context given bellow . include the time window, how many agents had events. how many error event during the time period and discribe errors:\n{current_context}"
        prompt = f"provide a detaled technical summery of the system for the system log in this given context given bellow . include the time window, how many agents had events. how many error event during the time period and discribe errors:\n{current_context}"
        #print(prompt,"UUUUUUUUUUUUUUUUUUUUUUUUUUUUUU")
        response=client.run(prompt)
        print(len(data),len(prompt),response['replies'])
        gptresponse=response['replies']
        data.append({"response": gptresponse, "prompt": prompt,"context":current_context})
        current_context=f"\n"
        time.sleep(10)
        row_count=0
    row_count=row_count+1
    
    #response=client.run(prompt)
    #print(len(data),response['replies'])
    #gptresponse=response['replies']
   # data.append({"response": gptresponse, "prompt": prompt,"context":context})
    #time.sleep(10)
       


0 30632 ['The system log captures multiple events of the platform_driveragent-4.0 and listeneragent-3.3 agents interacting and performing various tasks. The time window for the recorded events is from 2024-06-17 15:47:34 to 2024-06-17 15:48:30.\n\nDuring this time period, the platform_driveragent-4.0 agent had a total of 25 events logged, while the listeneragent-3.3 agent had a total of 25 events logged as well.\n\nAmong the events, there were several error events recorded by the platform_driveragent-4.0 agent. The errors were related to the `fakedriver` module within the `platform_driver` package. The errors indicated "Invalid default_value in EKGregister", and the system defaulted to using "sin(x)" instead.\n\nOverall, the system log shows the interaction and functioning of the platform_driveragent-4.0 and listeneragent-3.3 agents, with some errors being encountered and logged during the process.']
1 28835 ['In the given system log, the time window is from 2024-06-17 15:48:30 to 2024

KeyboardInterrupt: 

Chunk 1: This is a sample sentence
Chunk 2: sample sentence that we want
Chunk 3: we want to split into
Chunk 4: split into smaller chunks with
Chunk 5: chunks with an overlap


In [19]:
 df = pd.DataFrame(data)
 df.to_parquet('volttronlog_mass.parquet', index=False)

In [20]:
df = pd.read_parquet('volttronlog_mass.parquet')

# Display the DataFrame
print(df)

                                               response  \
0     [The system log captures multiple events of th...   
1     [In the given system log, the time window is f...   
2     [The system log contains events recorded betwe...   
3     [The given system log provides a series of eve...   
4     [The system log details events from multiple a...   
...                                                 ...   
1152  [In the given context, the system log details ...   
1153  [During the time window from 16:21:15 to 16:22...   
1154  [In the given log context, the system consists...   
1155  [During the time window from 16:23:45 to 16:25...   
1156  [Time window: June 18, 2024, 16:25:04 to June ...   

                                                 prompt  \
0     provide a detaled technical summery of the sys...   
1     provide a detaled technical summery of the sys...   
2     provide a detaled technical summery of the sys...   
3     provide a detaled technical summery of the sys...